In [ ]:
!pip install sktime
!pip install transformers


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.1/16.1 MB 10.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 16.4 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import torch
from itertools import zip_longest
import torch.nn as nn
from sklearn.ensemble import RandomForestClassifier
from sktime.transformations.panel.rocket import Rocket

In [ ]:
import scipy.io
from sklearn import svm
from sklearn.svm import SVC
from sklearn.decomposition import KernelPCA
import numpy as np
import math
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.svm import LinearSVC
from sklearn.multiclass import OneVsRestClassifier
import numpy as geek
import networkx as nx
from scipy.signal import butter, lfilter
from scipy import signal
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization, Embedding, Reshape
import keras
from random import shuffle
from keras.layers import LSTM, GRU
from keras import regularizers
from keras.layers import Layer
import keras.backend as K
import pandas as pd
from collections import Counter
from sklearn.model_selection import train_test_split
import statistics
from keras.callbacks import ModelCheckpoint
from keras.optimizers import SGD, Adam
import time
import numpy as np
from sklearn.linear_model import RidgeClassifierCV
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from keras.optimizers import Adam
## define onehot labeling function
def onehot_labels(labels):
    return np.eye(7)[labels]  

def get_one_hot(str_col:list):
    labels = []
    for st in str_col:
        labels.append(list(map(int, st.split(","))))
    return labels

def get_one_hot(str_col:list):
    labels = []
    for st in str_col:
        labels.append(list(map(int, st.split(","))))
    return labels

def normalization(signal):
    mean = statistics.mean(signal)
    std = statistics.stdev(signal)

    signal = [(x - mean) / std for x in signal]
    return signal

def get_X_tensors(df, segment_length=170):
    res = []
    for row in df.itertuples():
        tensors = []
        da_001 = list(row)[1: segment_length+1]
        da_001 = normalization(da_001)
        
        da_3 = list(row)[segment_length+1: 2*segment_length+1]
        da_3 = normalization(da_3)
        
        da_10 = list(row)[2*segment_length+1: 3*segment_length+1]
        da_10 = normalization(da_10)
        
        da3_ops = list(row)[3*segment_length+1: ]
        da3_ops = normalization(da3_ops)

        tensors.append(da_001)
        tensors.append(da_3)
        tensors.append(da_10)
        tensors.append(da3_ops)
        res.append(tensors)
    return res

def load_data(file_name='eyedata.csv'):
    df = pd.read_csv(file_name)
    return get_X_tensors(df.iloc[:, 1:], 170), get_one_hot(df.iloc[:, 0])

X, Y = load_data('/content/eyedata.csv')


labels = []
for label in Y:
    if label not in labels:
        labels.append(label)
        
Y_new = []
for y in Y:
    Y_new.append(labels.index(y))

np.random.seed(42)
X = np.array(X)
Y = np.array(Y_new)
permutation = np.random.permutation(len(X))
X = X[permutation]
Y = Y[permutation]

X_tr, X_val, Y_tr, Y_val = train_test_split(X, Y, test_size=0.1, random_state=10)
X_train, X_test, Y_train, Y_test = train_test_split(X_tr, Y_tr, test_size=0.11, random_state=10)

amplitude = np.max(X_train)/2 - np.min(X_train)/2
X_train_noisy_1 = X_train + np.random.normal(0, 0.001*amplitude, X_train.shape)
X_train_noisy_2 = X_train + np.random.normal(0, 0.004*amplitude, X_train.shape)
X_train_noisy_3 = X_train + np.random.normal(0, 0.008*amplitude, X_train.shape)
X_train_noisy_4 = X_train + np.random.normal(0, 0.012*amplitude, X_train.shape)
    
X_train = np.concatenate((X_train, X_train_noisy_1, X_train_noisy_2, X_train_noisy_3, X_train_noisy_4),0)
Y_train = np.concatenate((Y_train, Y_train, Y_train, Y_train, Y_train),0)

Y_train = onehot_labels(Y_train)
Y_val   = onehot_labels(Y_val)
Y_test  = onehot_labels(Y_test)
     

rocket, kernel_size = 2

In [ ]:
cnn = Sequential()
cnn.add(Conv1D(filters=16, kernel_size=2, activation='relu', input_shape=(X_train.shape[1], 1)))
cnn.add(MaxPooling1D(pool_size=2))
cnn.add(Flatten())
cnn.add(Dense(64, activation='relu'))
cnn.add(Dropout(0.4))
cnn.add(Dense(7, activation='softmax'))

adam = Adam(lr=0.001)
cnn.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])

rocket = Rocket() 
X_train_rkt = rocket.fit_transform(X_train)
X_val_rkt = rocket.transform(X_val)
X_test_rkt = rocket.transform(X_test)
clf = make_pipeline(StandardScaler(), RidgeClassifierCV(alphas=np.logspace(-3, 3, 10)))
clf.fit(X_train_rkt, Y_train)

score = clf.score(X_test_rkt, Y_test)
print('Test accuracy:', score)

/usr/local/lib/python3.9/dist-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Test accuracy: 0.7065868263473054


In [ ]:
cnn = Sequential()
cnn.add(Conv1D(filters=32, kernel_size=2, activation='relu', input_shape=(X_train.shape[1], 1)))
cnn.add(MaxPooling1D(pool_size=2))
cnn.add(Flatten())
cnn.add(Dense(64, activation='relu'))
cnn.add(Dropout(0.4))
cnn.add(Dense(7, activation='softmax'))

adam = Adam(lr=0.001)
cnn.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])

rocket = Rocket() 
X_train_rkt = rocket.fit_transform(X_train)
X_val_rkt = rocket.transform(X_val)
X_test_rkt = rocket.transform(X_test)
clf = make_pipeline(StandardScaler(), RidgeClassifierCV(alphas=np.logspace(-3, 3, 10)))
clf.fit(X_train_rkt, Y_train)

score = clf.score(X_test_rkt, Y_test)
print('Test accuracy:', score)

/usr/local/lib/python3.9/dist-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Test accuracy: 0.7125748502994012


In [ ]:
cnn = Sequential()
cnn.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(X_train.shape[1], 1)))
cnn.add(MaxPooling1D(pool_size=2))
cnn.add(Flatten())
cnn.add(Dense(64, activation='relu'))
cnn.add(Dropout(0.4))
cnn.add(Dense(7, activation='softmax'))

adam = Adam(lr=0.001)
cnn.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])

rocket = Rocket() 
X_train_rkt = rocket.fit_transform(X_train)
X_val_rkt = rocket.transform(X_val)
X_test_rkt = rocket.transform(X_test)
clf = make_pipeline(StandardScaler(), RidgeClassifierCV(alphas=np.logspace(-3, 3, 10)))
clf.fit(X_train_rkt, Y_train)

score = clf.score(X_test_rkt, Y_test)
print('Test accuracy:', score)

/usr/local/lib/python3.9/dist-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Test accuracy: 0.7245508982035929


In [ ]:
cnn = Sequential()
cnn.add(Conv1D(filters=128, kernel_size=2, activation='relu', input_shape=(X_train.shape[1], 1)))
cnn.add(MaxPooling1D(pool_size=2))
cnn.add(Flatten())
cnn.add(Dense(64, activation='relu'))
cnn.add(Dropout(0.4))
cnn.add(Dense(7, activation='softmax'))

adam = Adam(lr=0.001)
cnn.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])

rocket = Rocket() 
X_train_rkt = rocket.fit_transform(X_train)
X_val_rkt = rocket.transform(X_val)
X_test_rkt = rocket.transform(X_test)
clf = make_pipeline(StandardScaler(), RidgeClassifierCV(alphas=np.logspace(-3, 3, 10)))
clf.fit(X_train_rkt, Y_train)

score = clf.score(X_test_rkt, Y_test)
print('Test accuracy:', score)

/usr/local/lib/python3.9/dist-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Test accuracy: 0.7245508982035929


rocket, kernel_size = 3

In [ ]:
cnn = Sequential()
cnn.add(Conv1D(filters=16, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)))
cnn.add(MaxPooling1D(pool_size=2))
cnn.add(Flatten())
cnn.add(Dense(64, activation='relu'))
cnn.add(Dropout(0.4))
cnn.add(Dense(7, activation='softmax'))

adam = Adam(lr=0.001)
cnn.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])

rocket = Rocket() 
X_train_rkt = rocket.fit_transform(X_train)
X_val_rkt = rocket.transform(X_val)
X_test_rkt = rocket.transform(X_test)
clf = make_pipeline(StandardScaler(), RidgeClassifierCV(alphas=np.logspace(-3, 3, 10)))
clf.fit(X_train_rkt, Y_train)

score = clf.score(X_test_rkt, Y_test)
print('Test accuracy:', score)

/usr/local/lib/python3.9/dist-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Test accuracy: 0.7125748502994012


In [ ]:
cnn = Sequential()
cnn.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)))
cnn.add(MaxPooling1D(pool_size=2))
cnn.add(Flatten())
cnn.add(Dense(64, activation='relu'))
cnn.add(Dropout(0.4))
cnn.add(Dense(7, activation='softmax'))

adam = Adam(lr=0.001)
cnn.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])

rocket = Rocket() 
X_train_rkt = rocket.fit_transform(X_train)
X_val_rkt = rocket.transform(X_val)
X_test_rkt = rocket.transform(X_test)
clf = make_pipeline(StandardScaler(), RidgeClassifierCV(alphas=np.logspace(-3, 3, 10)))
clf.fit(X_train_rkt, Y_train)

score = clf.score(X_test_rkt, Y_test)
print('Test accuracy:', score)

/usr/local/lib/python3.9/dist-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Test accuracy: 0.6946107784431138


In [ ]:
cnn = Sequential()
cnn.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)))
cnn.add(MaxPooling1D(pool_size=2))
cnn.add(Flatten())
cnn.add(Dense(64, activation='relu'))
cnn.add(Dropout(0.4))
cnn.add(Dense(7, activation='softmax'))

adam = Adam(lr=0.001)
cnn.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])

rocket = Rocket() 
X_train_rkt = rocket.fit_transform(X_train)
X_val_rkt = rocket.transform(X_val)
X_test_rkt = rocket.transform(X_test)
clf = make_pipeline(StandardScaler(), RidgeClassifierCV(alphas=np.logspace(-3, 3, 10)))
clf.fit(X_train_rkt, Y_train)

score = clf.score(X_test_rkt, Y_test)
print('Test accuracy:', score)

/usr/local/lib/python3.9/dist-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Test accuracy: 0.6946107784431138


In [ ]:
cnn = Sequential()
cnn.add(Conv1D(filters=128, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)))
cnn.add(MaxPooling1D(pool_size=2))
cnn.add(Flatten())
cnn.add(Dense(64, activation='relu'))
cnn.add(Dropout(0.4))
cnn.add(Dense(7, activation='softmax'))

adam = Adam(lr=0.001)
cnn.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])

rocket = Rocket() 
X_train_rkt = rocket.fit_transform(X_train)
X_val_rkt = rocket.transform(X_val)
X_test_rkt = rocket.transform(X_test)
clf = make_pipeline(StandardScaler(), RidgeClassifierCV(alphas=np.logspace(-3, 3, 10)))
clf.fit(X_train_rkt, Y_train)

score = clf.score(X_test_rkt, Y_test)
print('Test accuracy:', score)

/usr/local/lib/python3.9/dist-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Test accuracy: 0.6826347305389222


ROCKET+random forest, kernel_size = 2

In [ ]:
cnn = Sequential()
cnn.add(Conv1D(filters=16, kernel_size=2, activation='relu', input_shape=(X_train.shape[1], 1)))
cnn.add(MaxPooling1D(pool_size=2))
cnn.add(Flatten())
cnn.add(Dense(64, activation='relu'))
cnn.add(Dropout(0.4))
cnn.add(Dense(7, activation='softmax'))

adam = Adam(lr=0.001)
cnn.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])

rocket, rfc = Rocket(), RandomForestClassifier()

X_train_rkt = rocket.fit_transform(X_train)
X_val_rkt = rocket.transform(X_val)
X_test_rkt = rocket.transform(X_test)
clf = make_pipeline(StandardScaler(), RidgeClassifierCV(alphas=np.logspace(-3, 3, 10)))
clf.fit(X_train_rkt, Y_train)

score = clf.score(X_test_rkt, Y_test)
print('Test accuracy:', score)

/usr/local/lib/python3.9/dist-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


ValueError: ignored

In [ ]:
cnn = Sequential()
cnn.add(Conv1D(filters=32, kernel_size=2, activation='relu', input_shape=(X_train.shape[1], 1)))
cnn.add(MaxPooling1D(pool_size=2))
cnn.add(Flatten())
cnn.add(Dense(64, activation='relu'))
cnn.add(Dropout(0.4))
cnn.add(Dense(7, activation='softmax'))

adam = Adam(lr=0.001)
cnn.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])

rocket, rfc = Rocket(), RandomForestClassifier()

X_train_rkt = rocket.fit_transform(X_train)
X_val_rkt = rocket.transform(X_val)
X_test_rkt = rocket.transform(X_test)
clf = make_pipeline(StandardScaler(), RidgeClassifierCV(alphas=np.logspace(-3, 3, 10)))
clf.fit(X_train_rkt, Y_train)

score = clf.score(X_test_rkt, Y_test)
print('Test accuracy:', score)

In [ ]:
cnn = Sequential()
cnn.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(X_train.shape[1], 1)))
cnn.add(MaxPooling1D(pool_size=2))
cnn.add(Flatten())
cnn.add(Dense(64, activation='relu'))
cnn.add(Dropout(0.4))
cnn.add(Dense(7, activation='softmax'))

adam = Adam(lr=0.001)
cnn.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])

rocket, rfc = Rocket(), RandomForestClassifier()

X_train_rkt = rocket.fit_transform(X_train)
X_val_rkt = rocket.transform(X_val)
X_test_rkt = rocket.transform(X_test)
clf = make_pipeline(StandardScaler(), RidgeClassifierCV(alphas=np.logspace(-3, 3, 10)))
clf.fit(X_train_rkt, Y_train)

score = clf.score(X_test_rkt, Y_test)
print('Test accuracy:', score)

In [ ]:
cnn = Sequential()
cnn.add(Conv1D(filters=128, kernel_size=2, activation='relu', input_shape=(X_train.shape[1], 1)))
cnn.add(MaxPooling1D(pool_size=2))
cnn.add(Flatten())
cnn.add(Dense(64, activation='relu'))
cnn.add(Dropout(0.4))
cnn.add(Dense(7, activation='softmax'))

adam = Adam(lr=0.001)
cnn.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])

rocket, rfc = Rocket(), RandomForestClassifier()

X_train_rkt = rocket.fit_transform(X_train)
X_val_rkt = rocket.transform(X_val)
X_test_rkt = rocket.transform(X_test)
clf = make_pipeline(StandardScaler(), RidgeClassifierCV(alphas=np.logspace(-3, 3, 10)))
clf.fit(X_train_rkt, Y_train)

score = clf.score(X_test_rkt, Y_test)
print('Test accuracy:', score)

ROCKET+random forest, kernel_size = 3

In [ ]:
cnn = Sequential()
cnn.add(Conv1D(filters=16, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)))
cnn.add(MaxPooling1D(pool_size=2))
cnn.add(Flatten())
cnn.add(Dense(64, activation='relu'))
cnn.add(Dropout(0.4))
cnn.add(Dense(7, activation='softmax'))

adam = Adam(lr=0.001)
cnn.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])

rocket, rfc = Rocket(), RandomForestClassifier()

X_train_rkt = rocket.fit_transform(X_train)
X_val_rkt = rocket.transform(X_val)
X_test_rkt = rocket.transform(X_test)
clf = make_pipeline(StandardScaler(), RidgeClassifierCV(alphas=np.logspace(-3, 3, 10)))
clf.fit(X_train_rkt, Y_train)

score = clf.score(X_test_rkt, Y_test)
print('Test accuracy:', score)

In [ ]:
cnn = Sequential()
cnn.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)))
cnn.add(MaxPooling1D(pool_size=2))
cnn.add(Flatten())
cnn.add(Dense(64, activation='relu'))
cnn.add(Dropout(0.4))
cnn.add(Dense(7, activation='softmax'))

adam = Adam(lr=0.001)
cnn.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])

rocket, rfc = Rocket(), RandomForestClassifier()

X_train_rkt = rocket.fit_transform(X_train)
X_val_rkt = rocket.transform(X_val)
X_test_rkt = rocket.transform(X_test)
clf = make_pipeline(StandardScaler(), RidgeClassifierCV(alphas=np.logspace(-3, 3, 10)))
clf.fit(X_train_rkt, Y_train)

score = clf.score(X_test_rkt, Y_test)
print('Test accuracy:', score)

In [ ]:
cnn = Sequential()
cnn.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)))
cnn.add(MaxPooling1D(pool_size=2))
cnn.add(Flatten())
cnn.add(Dense(64, activation='relu'))
cnn.add(Dropout(0.4))
cnn.add(Dense(7, activation='softmax'))

adam = Adam(lr=0.001)
cnn.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])

rocket, rfc = Rocket(), RandomForestClassifier()

X_train_rkt = rocket.fit_transform(X_train)
X_val_rkt = rocket.transform(X_val)
X_test_rkt = rocket.transform(X_test)
clf = make_pipeline(StandardScaler(), RidgeClassifierCV(alphas=np.logspace(-3, 3, 10)))
clf.fit(X_train_rkt, Y_train)

score = clf.score(X_test_rkt, Y_test)
print('Test accuracy:', score)

In [ ]:
cnn = Sequential()
cnn.add(Conv1D(filters=128, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)))
cnn.add(MaxPooling1D(pool_size=2))
cnn.add(Flatten())
cnn.add(Dense(64, activation='relu'))
cnn.add(Dropout(0.4))
cnn.add(Dense(7, activation='softmax'))

adam = Adam(lr=0.001)
cnn.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])

rocket, rfc = Rocket(), RandomForestClassifier()

X_train_rkt = rocket.fit_transform(X_train)
X_val_rkt = rocket.transform(X_val)
X_test_rkt = rocket.transform(X_test)
clf = make_pipeline(StandardScaler(), RidgeClassifierCV(alphas=np.logspace(-3, 3, 10)))
clf.fit(X_train_rkt, Y_train)

score = clf.score(X_test_rkt, Y_test)
print('Test accuracy:', score)